In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the datasets
# Load the datasets
customers = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/customers_final.csv')
engagements = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/engagements_final.csv')
marketing = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/marketing_final.csv')
transactions = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1_2024/main/Datasets/transactions_final.csv')

# Merge the datasets on customer_id
merged_data = customers.merge(transactions, on='customer_id', how='left') \
                       .merge(engagements, on='customer_id', how='left') \
                       .merge(marketing, on='customer_id', how='left')

# Handle missing values
median_age = merged_data['age'].median()
merged_data['age'].fillna(median_age, inplace=True)

most_frequent_gender = merged_data['gender'].mode()[0]
merged_data['gender'].fillna(most_frequent_gender, inplace=True)

# Convert join_date and last_purchase_date to datetime
merged_data['join_date'] = pd.to_datetime(merged_data['join_date'])
merged_data['last_purchase_date'] = pd.to_datetime(merged_data['last_purchase_date'])

# Calculate customer usage lifespan in months
merged_data['usage_lifespan'] = ((merged_data['last_purchase_date'] - merged_data['join_date']).dt.days) / 30

# Filter out new customers (usage lifespan less than 1 month)
merged_data = merged_data[merged_data['usage_lifespan'] >= 1]

# Calculate total transaction amount for each customer
total_transactions = merged_data.groupby('customer_id')['transaction_amount'].sum().reset_index()
total_transactions.columns = ['customer_id', 'total_transaction_amount']

# Merge total transaction amount back into the main dataset
merged_data = merged_data.merge(total_transactions, on='customer_id', how='left')

# Calculate average spending per month
merged_data['avg_spending_per_month'] = merged_data['total_transaction_amount'] / merged_data['usage_lifespan']

# Calculate CLV
merged_data['CLV'] = merged_data['avg_spending_per_month'] * merged_data['usage_lifespan']

# Drop duplicate rows to keep one row per customer
merged_data = merged_data.drop_duplicates(subset=['customer_id'])

# Display the first few rows of the updated dataset
updated_clv_data = merged_data[['customer_id', 'join_date', 'last_purchase_date', 'usage_lifespan', 'avg_spending_per_month', 'CLV']].head()
print(updated_clv_data)

# Select relevant features excluding total_transactions, avg_transaction_amount, and recency
X = merged_data[['age', 'gender', 'number_of_site_visits', 'number_of_emails_opened', 'number_of_clicks']]
X = pd.get_dummies(X, columns=['gender'], drop_first=True)

y = merged_data['CLV']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}, RMSE: {rmse}, R^2: {r2}")


/var/folders/03/qxtr7cc92pg1c6lvx8kqq6f80000gn/T/ipykernel_61734/3933125191.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['age'].fillna(median_age, inplace=True)
/var/folders/03/qxtr7cc92pg1c6lvx8kqq6f80000gn/T/ipykernel_61734/3933125191.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values

     customer_id  join_date last_purchase_date  usage_lifespan  \
0              1 2023-11-20         2024-03-17        3.933333   
24             2 2021-09-08         2023-10-25       25.900000   
60             3 2021-06-01         2022-11-27       18.133333   
72             4 2022-01-01         2022-09-01        8.100000   
152            5 2022-01-24         2023-06-02       16.466667   

     avg_spending_per_month       CLV  
0               3568.962712  14037.92  
24               939.200000  24325.28  
60               160.463603   2909.74  
72              3888.730864  31498.72  
152             3771.145749  62098.20  
MAE: 12347.14445589729, RMSE: 15082.483564560343, R^2: -0.0014510429340133246
